<a href="https://colab.research.google.com/github/venu-analytics/Analytics-Projects/blob/main/Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initialisation

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.2.1/spark-2.2.1-bin-hadoop2.7.tgz
!tar xf spark-2.2.1-bin-hadoop2.7.tgz
!pip install -q findspark

Hit:1 http://security.ubuntu.com/ubuntu artful-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu artful InRelease
Hit:3 http://archive.ubuntu.com/ubuntu artful-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu artful-backports InRelease
Reading package lists... Done


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.2.1-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

## Loading the data

In [ ]:
import urllib
f = urllib.request.urlretrieve ("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "kddcup.data_10_percent.gz")

In [ ]:
data_file = "./kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)

print ("Train data size is {}".format(raw_data.count()))

Train data size is 494021


In [ ]:
ft = urllib.request.urlretrieve("http://kdd.ics.uci.edu/databases/kddcup99/corrected.gz", "corrected.gz")

In [ ]:
test_data_file = "./corrected.gz"
test_raw_data = sc.textFile(test_data_file)

print ("Test data size is {}".format(test_raw_data.count()))

Test data size is 311029


## Preparing the Training Data

In [ ]:
from pyspark.mllib.regression import LabeledPoint
from numpy import array

def parse_interaction(line):
    line_split = line.split(",")
    # leave_out = [1,2,3,41]
    clean_line_split = line_split[0:1]+line_split[4:41]
    attack = 1.0
    if line_split[41]=='normal.':
        attack = 0.0
    return LabeledPoint(attack, array([float(x) for x in clean_line_split]))

training_data = raw_data.map(parse_interaction)

In [ ]:
test_data = test_raw_data.map(parse_interaction)

## Training a Classifier

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from time import time

# Build the model
t0 = time()
logit_model = LogisticRegressionWithLBFGS.train(training_data)
tt = time() - t0

print ("Classifier trained in {} seconds".format(round(tt,3)))

Classifier trained in 69.584 seconds


## Evaluating the model on new data

In [ ]:
labels_and_preds = test_data.map(lambda p: (p.label, logit_model.predict(p.features)))

In [ ]:
t0 = time()
test_accuracy = labels_and_preds.filter(lambda x : x[0] == x[1]).count() / float(test_data.count())
tt = time() - t0

print ("Prediction made in ",round(tt,3)," seconds. Test accuracy is ",round(test_accuracy,4))

Prediction made in  13.403  seconds. Test accuracy is  0.9049


In [ ]:
def parse_interaction_corr(line):
    line_split = line.split(",")
    # leave_out = [1,2,3,25,27,35,38,40,41]
    clean_line_split = line_split[0:1]+line_split[4:25]+line_split[26:27]+line_split[28:35]+line_split[36:38]+line_split[39:40]
    attack = 1.0
    if line_split[41]=='normal.':
        attack = 0.0
    return LabeledPoint(attack, array([float(x) for x in clean_line_split]))

corr_reduced_training_data = raw_data.map(parse_interaction_corr)
corr_reduced_test_data = test_raw_data.map(parse_interaction_corr)

In [ ]:
# Build the model
t0 = time()
logit_model_2 = LogisticRegressionWithLBFGS.train(corr_reduced_training_data)
tt = time() - t0

print ("Classifier trained in ",round(tt,3)," seconds")

Classifier trained in  59.099  seconds


In [ ]:
t0 = time()
labels_and_preds = corr_reduced_test_data.map(lambda p: (p.label, logit_model_2.predict(p.features)))
tt = time() - t0
test_accuracy = labels_and_preds.filter(lambda x: x[0] == x[1]).count() / float(corr_reduced_test_data.count())


print ("Prediction made in ",round(tt,3),"seconds. Test accuracy is ",round(test_accuracy,4)) 

Prediction made in  0.0 seconds. Test accuracy is  0.8158


## Using Hypothesis

In [ ]:
feature_names = ["land","wrong_fragment",
             "urgent","hot","num_failed_logins","logged_in","num_compromised",
             "root_shell","su_attempted","num_root","num_file_creations",
             "num_shells","num_access_files","num_outbound_cmds",
             "is_hot_login","is_guest_login","count","srv_count","serror_rate",
             "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
             "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
             "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
             "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
             "dst_host_rerror_rate","dst_host_srv_rerror_rate"]

In [ ]:
def parse_interaction_categorical(line):
    line_split = line.split(",")
    clean_line_split = line_split[6:41]
    attack = 1.0
    if line_split[41]=='normal.':
        attack = 0.0
    return LabeledPoint(attack, array([float(x) for x in clean_line_split]))

In [ ]:
training_data_categorical = raw_data.map(parse_interaction_categorical)

In [ ]:
training_data_categorical.take(3)

[LabeledPoint(0.0, [0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,8.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9.0,9.0,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0]),
 LabeledPoint(0.0, [0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,8.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,19.0,19.0,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0]),
 LabeledPoint(0.0, [0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,8.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,29.0,29.0,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0])]

In [ ]:
from pyspark.mllib.stat import Statistics

chi = Statistics.chiSqTest(training_data_categorical)

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', 30)

records = [(result.statistic, result.pValue) for result in chi]

chi_df = pd.DataFrame(data=records, index= feature_names, columns=["Statistic","p-value"])

chi_df

,Statistic,p-value
land,3.191410,7.402616e-02
wrong_fragment,304.309631,0.000000e+00
urgent,4.814020,1.859331e-01
hot,1939.879629,0.000000e+00
num_failed_logins,24.935016,1.434174e-04
logged_in,312455.306674,0.000000e+00
num_compromised,635.549407,0.000000e+00
root_shell,17.030507,3.678404e-05
su_attempted,39.839347,2.233550e-09
num_root,2277.286802,0.000000e+00


In [ ]:
def parse_interaction_chi(line):
    line_split = line.split(",")
    # leave_out = [1,2,3,6,19,41]
    clean_line_split = line_split[0:1] + line_split[4:6] + line_split[7:19] + line_split[20:41]
    attack = 1.0
    if line_split[41]=='normal.':
        attack = 0.0
    return LabeledPoint(attack, array([float(x) for x in clean_line_split]))

In [ ]:
training_data_chi = raw_data.map(parse_interaction_chi)
test_data_chi = test_raw_data.map(parse_interaction_chi)

In [ ]:
# Build the model
t0 = time()
logit_model_chi = LogisticRegressionWithLBFGS.train(training_data_chi)
tt = time() - t0

print ("Classifier trained in ",round(tt,3)," seconds")

Classifier trained in  62.37  seconds


In [ ]:
t0 = time()
labels_and_preds = test_data_chi.map(lambda p: (p.label, logit_model_chi.predict(p.features)))
tt = time() - t0
test_accuracy = labels_and_preds.filter(lambda x: x[0] == x[1]).count() / float(test_data_chi.count())

print ("Prediction made in ",round(tt,3)," seconds. Test accuracy is ", round(test_accuracy,4))

Prediction made in  0.0  seconds. Test accuracy is  0.9031
